In [2]:
# =============================================================================
# BLOCK 1: SETUP, IMPORTS, AND DATA LOADING
# =============================================================================
import warnings
warnings.filterwarnings('ignore')
import time
import pandas as pd
import numpy as np
import gc
from sklearn.model_selection import StratifiedKFold, train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_squared_error
import xgboost as xgb
import optuna

print("--- BLOCK 1: SETUP ---")
print("# Libraries imported successfully.")

# Helper Function for Winkler Score
def winkler_score(y_true, lower, upper, alpha=0.1, return_coverage=False):
    width = upper - lower
    penalty_lower = np.where(y_true < lower, (2 / alpha) * (lower - y_true), 0)
    penalty_upper = np.where(y_true > upper, (2 / alpha) * (y_true - upper), 0)
    score = width + penalty_lower + penalty_upper
    if return_coverage:
        coverage = np.mean((y_true >= lower) & (y_true <= upper))
        return np.mean(score), coverage
    return np.mean(score)

# Global Constants
N_SPLITS = 5
RANDOM_STATE = 42
DATA_PATH = './'
N_OPTUNA_TRIALS = 30
COMPETITION_ALPHA = 0.1
print("# Global constants defined.")

# Load Raw Data
try:
    drop_cols=['id', 'golf', 'view_rainier', 'view_skyline', 'view_lakesamm', 'view_otherwater', 'view_other']
    df_train = pd.read_csv(DATA_PATH + 'dataset.csv').drop(columns=drop_cols)
    df_test = pd.read_csv(DATA_PATH + 'test.csv').drop(columns=drop_cols)
    print("# Raw data loaded successfully.")
except FileNotFoundError:
    print("ERROR: Could not find 'dataset.csv' or 'test.csv'.")
    exit()

# Prepare Target Variable
y_true = df_train['sale_price'].copy()
print("# Target variable 'y_true' created.")
print("# Setup complete.")
print("-" * 50)

--- BLOCK 1: SETUP ---
# Libraries imported successfully.
# Global constants defined.
# Raw data loaded successfully.
# Target variable 'y_true' created.
# Setup complete.
--------------------------------------------------


In [3]:
# =============================================================================
# BLOCK 2: FEATURE ENGINEERING
# =============================================================================
print("\n--- BLOCK 2: FEATURE ENGINEERING ---")
print("# This block creates all features using the winning 'mean-error' strategy's feature set.")

# Combine for consistent processing
df_train['is_train'] = 1
df_test['is_train'] = 0
train_ids = df_train.index
test_ids = df_test.index
all_data = pd.concat([df_train, df_test], axis=0).reset_index(drop=True)

# A) Brute-Force Numerical Interactions
print("# Creating brute-force numerical interaction features...")
NUMS = ['area', 'land_val', 'imp_val', 'sqft_lot', 'sqft', 'sqft_1', 'grade', 'year_built']
for i in range(len(NUMS)):
    for j in range(i + 1, len(NUMS)):
        all_data[f'{NUMS[i]}_x_{NUMS[j]}'] = all_data[NUMS[i]] * all_data[NUMS[j]]

# B) Date Features
all_data['sale_date'] = pd.to_datetime(all_data['sale_date'])
all_data['year'] = all_data['sale_date'].dt.year
all_data['month'] = all_data['sale_date'].dt.month
all_data['year_diff'] = all_data['year'] - all_data['year_built']

# C) TF-IDF Text Features
print("# Creating TF-IDF features for text columns...")
text_cols = ['subdivision', 'zoning', 'city', 'sale_warning', 'join_status', 'submarket']
all_data[text_cols] = all_data[text_cols].fillna('missing').astype(str)
for col in text_cols:
    tfidf = TfidfVectorizer(analyzer='char', ngram_range=(3, 5), max_features=128, binary=True)
    tfidf_matrix = tfidf.fit_transform(all_data[col])
    svd = TruncatedSVD(n_components=8, random_state=RANDOM_STATE)
    tfidf_svd = svd.fit_transform(tfidf_matrix)
    tfidf_df = pd.DataFrame(tfidf_svd, columns=[f'{col}_tfidf_svd_{i}' for i in range(8)])
    all_data = pd.concat([all_data, tfidf_df], axis=1)

# D) Log transform some interactions
for c in ['land_val_x_imp_val', 'land_val_x_sqft', 'imp_val_x_sqft']:
    if c in all_data.columns:
        all_data[c] = np.log1p(all_data[c].fillna(0))
        
# E) Final Cleanup
print("# Finalizing feature set...")
cols_to_drop = ['sale_date', 'subdivision', 'zoning', 'city', 'sale_warning', 'join_status', 'submarket']
all_data = all_data.drop(columns=cols_to_drop)
all_data.fillna(0, inplace=True)

# Separate final datasets
X = all_data[all_data['is_train'] == 1].drop(columns=['is_train', 'sale_price'])
X_test = all_data[all_data['is_train'] == 0].drop(columns=['is_train', 'sale_price'])
X.index, X_test.index = train_ids, test_ids
X_test = X_test[X.columns]
print(f"# Feature engineering complete. Total features: {X.shape[1]}")
del all_data
gc.collect()
print("-" * 50)


--- BLOCK 2: FEATURE ENGINEERING ---
# This block creates all features using the winning 'mean-error' strategy's feature set.
# Creating brute-force numerical interaction features...
# Creating TF-IDF features for text columns...
# Finalizing feature set...
# Feature engineering complete. Total features: 111
--------------------------------------------------


In [4]:
# =============================================================================
# BLOCK 3: TWO-STAGE MODELING, TUNING, AND SUBMISSION
# =============================================================================
print("\n--- BLOCK 3: TWO-STAGE MODELING PIPELINE ---")

# --- STAGE 1, PART 1: Tuning Mean Prediction Model ---
print("\n# STAGE 1, PART 1: Tuning Mean Prediction Model...")
print("# EXPLANATION: Using Optuna to find the best XGBoost settings to predict the mean price.")
def objective_mean(trial):
    train_x, val_x, train_y, val_y = train_test_split(X, y_true, test_size=0.2, random_state=RANDOM_STATE)
    params = {
        'objective': 'reg:squarederror', 'eval_metric': 'rmse', 'tree_method': 'hist',
        'eta': trial.suggest_float('eta', 0.01, 0.08, log=True),
        'max_depth': trial.suggest_int('max_depth', 6, 14),
        'subsample': trial.suggest_float('subsample', 0.7, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.7, 1.0),
        'lambda': trial.suggest_float('lambda', 1e-4, 10.0, log=True),
        'alpha': trial.suggest_float('alpha', 1e-4, 10.0, log=True),
    }
    model = xgb.XGBRegressor(**params, n_estimators=1500, random_state=RANDOM_STATE, n_jobs=-1, early_stopping_rounds=50)
    model.fit(train_x, train_y, eval_set=[(val_x, val_y)], verbose=False)
    preds = model.predict(val_x)
    return np.sqrt(mean_squared_error(val_y, preds))

study_mean = optuna.create_study(direction='minimize')
study_mean.optimize(objective_mean, n_trials=N_OPTUNA_TRIALS)
best_params_mean = study_mean.best_params
print(f"# Mean Model Tuning Complete. Best Validation RMSE: ${study_mean.best_value:,.2f}")

# --- STAGE 1, PART 2: K-Fold Training of Mean Model ---
print("\n# STAGE 1, PART 2: K-Fold Training of Mean Model...")
print("# EXPLANATION: Using the best settings to train 5 models for robust mean predictions.")
skf = StratifiedKFold(n_splits=N_SPLITS, shuffle=True, random_state=RANDOM_STATE)
oof_mean_preds = np.zeros(len(X))
test_mean_preds = np.zeros(len(X_test))
for fold, (train_idx, val_idx) in enumerate(skf.split(X, df_train['grade'])):
    print(f"  Mean Model - Fold {fold+1}/{N_SPLITS}...")
    model = xgb.XGBRegressor(**best_params_mean, n_estimators=2000, objective='reg:squarederror', eval_metric='rmse', tree_method='hist', random_state=RANDOM_STATE, n_jobs=-1, early_stopping_rounds=100)
    model.fit(X.iloc[train_idx], y_true.iloc[train_idx], eval_set=[(X.iloc[val_idx], y_true.iloc[val_idx])], verbose=False)
    oof_mean_preds[val_idx] = model.predict(X.iloc[val_idx])
    test_mean_preds += model.predict(X_test) / N_SPLITS

# --- STAGE 2: K-Fold Training of Error Model ---
print("\n# STAGE 2: K-Fold Training of Error Model...")
print("# EXPLANATION: Training a second model to predict the size of the first model's errors.")
error_target = np.abs(y_true - oof_mean_preds)
X_for_error = X.copy()
X_for_error['mean_pred_oof'] = oof_mean_preds
X_test_for_error = X_test.copy()
X_test_for_error['mean_pred_oof'] = test_mean_preds
params_error = {'objective': 'reg:squarederror', 'eval_metric': 'rmse', 'tree_method': 'hist', 'eta': 0.03, 'max_depth': 7, 'random_state': RANDOM_STATE}
oof_error_preds = np.zeros(len(X))
test_error_preds = np.zeros(len(X_test))
for fold, (train_idx, val_idx) in enumerate(skf.split(X_for_error, df_train['grade'])):
    print(f"  Error Model - Fold {fold+1}/{N_SPLITS}...")
    model = xgb.XGBRegressor(**params_error, n_estimators=1500, n_jobs=-1, early_stopping_rounds=100)
    model.fit(X_for_error.iloc[train_idx], error_target.iloc[train_idx], eval_set=[(X_for_error.iloc[val_idx], error_target.iloc[val_idx])], verbose=False)
    oof_error_preds[val_idx] = model.predict(X_for_error.iloc[val_idx])
    test_error_preds += model.predict(X_test_for_error) / N_SPLITS



[I 2025-07-06 15:11:47,004] A new study created in memory with name: no-name-a51abc5a-1355-4a5b-bf19-7c412cccc1f3



--- BLOCK 3: TWO-STAGE MODELING PIPELINE ---

# STAGE 1, PART 1: Tuning Mean Prediction Model...
# EXPLANATION: Using Optuna to find the best XGBoost settings to predict the mean price.


[I 2025-07-06 15:12:01,651] Trial 0 finished with value: 101513.52288242192 and parameters: {'eta': 0.05277751877165471, 'max_depth': 6, 'subsample': 0.9502685680203791, 'colsample_bytree': 0.8594411179712, 'lambda': 0.0014597297975626454, 'alpha': 0.00019263141276771274}. Best is trial 0 with value: 101513.52288242192.
[I 2025-07-06 15:12:59,480] Trial 1 finished with value: 101451.42143903159 and parameters: {'eta': 0.010378702349099324, 'max_depth': 11, 'subsample': 0.7663025711400787, 'colsample_bytree': 0.7430461143646169, 'lambda': 5.195518231489485, 'alpha': 0.0034817142150336384}. Best is trial 1 with value: 101451.42143903159.
[I 2025-07-06 15:13:40,679] Trial 2 finished with value: 105021.21629461354 and parameters: {'eta': 0.015305674435956491, 'max_depth': 6, 'subsample': 0.9874705575030425, 'colsample_bytree': 0.8644345656996169, 'lambda': 0.03689287276806164, 'alpha': 0.631069603949334}. Best is trial 1 with value: 101451.42143903159.
[I 2025-07-06 15:15:12,962] Trial 3 f

# Mean Model Tuning Complete. Best Validation RMSE: $99,464.64

# STAGE 1, PART 2: K-Fold Training of Mean Model...
# EXPLANATION: Using the best settings to train 5 models for robust mean predictions.
  Mean Model - Fold 1/5...
  Mean Model - Fold 2/5...
  Mean Model - Fold 3/5...
  Mean Model - Fold 4/5...
  Mean Model - Fold 5/5...

# STAGE 2: K-Fold Training of Error Model...
# EXPLANATION: Training a second model to predict the size of the first model's errors.
  Error Model - Fold 1/5...
  Error Model - Fold 2/5...
  Error Model - Fold 3/5...
  Error Model - Fold 4/5...
  Error Model - Fold 5/5...

# FINAL STAGE: Asymmetric Calibration & Submission...
# EXPLANATION: Finding the best 'stretch' multipliers (a and b) for our intervals to hit 90% coverage.

# Grid search complete.
# Final OOF Winkler Score: 305,277.21
# Best multipliers found: a (lower)=1.98, b (upper)=2.18


In [9]:
# --- FINAL ASYMMETRIC CALIBRATION AND SUBMISSION ---
print("\n# FINAL STAGE: Asymmetric Calibration & Submission...")
print("# EXPLANATION: Finding the best 'stretch' multipliers (a and b) for our intervals to hit 90% coverage.")
oof_error_final = np.clip(oof_error_preds, 0, None)
best_a, best_b, best_metric = 2.0, 2.0, float('inf')
for a in np.arange(1.90, 2.31, 0.01):
    for b in np.arange(2.10, 2.51, 0.01):
        low = oof_mean_preds - oof_error_final * a
        high = oof_mean_preds + oof_error_final * b
        metric, coverage = winkler_score(y_true, low, high, alpha=COMPETITION_ALPHA, return_coverage=True)
        if metric < best_metric:
            best_metric = metric
            best_a, best_b = a, b

print(f"\n# Grid search complete.")
print(f"# Final OOF Winkler Score: {best_metric:,.2f}")
print(f"# Best multipliers found: a (lower)={best_a:.2f}, b (upper)={best_b:.2f}")


# FINAL STAGE: Asymmetric Calibration & Submission...
# EXPLANATION: Finding the best 'stretch' multipliers (a and b) for our intervals to hit 90% coverage.

# Grid search complete.
# Final OOF Winkler Score: 305,277.21
# Best multipliers found: a (lower)=1.98, b (upper)=2.18


In [10]:
# =============================================================================
# FINAL SUBMISSION BLOCK
# =============================================================================
print("--- Creating Final Submission File ---")
print(f"# This will use the models and calibration factors just trained.")

# --- 1. Load the Original Test IDs ---
# This is the safest way to ensure the submission IDs are correct.
try:
    original_test_ids = pd.read_csv('./test.csv', usecols=['id'])['id']
    print("# Original test IDs loaded successfully.")
except FileNotFoundError:
    print("ERROR: test.csv not found. Please ensure it's in the correct directory.")
    exit()

# --- 2. Use the live variables from the previous cell ---
# The variables 'test_mean_preds', 'test_error_preds', 'best_a', and 'best_b'
# should all be in memory from the long run you just completed.
print(f"# Applying best found multipliers: a={best_a:.2f}, b={best_b:.2f}")

# The predictions are already on the correct dollar scale.
# We just need to clip the error predictions to be non-negative.
test_mean_final = test_mean_preds
test_error_final = np.clip(test_error_preds, 0, None)

# --- 3. Construct the Final Calibrated Intervals ---
final_lower = test_mean_final - test_error_final * best_a
final_upper = test_mean_final + test_error_final * best_b

# Final safety checks to ensure lower <= upper and prices are plausible.
final_upper = np.maximum(final_lower, final_upper)
# We can clip at a reasonable minimum price, e.g., $1000
final_lower = np.clip(final_lower, 1000, None)
final_upper = np.clip(final_upper, 1000, None)

# --- 4. Create and Save the Submission DataFrame ---
submission_df = pd.DataFrame({
    'id': original_test_ids,
    'pi_lower': final_lower,
    'pi_upper': final_upper
})

submission_df.to_csv('submission_final_synthesis.csv', index=False)
print("\n'submission_final_synthesis.csv' created successfully!")
display(submission_df.head())

--- Creating Final Submission File ---
# This will use the models and calibration factors just trained.
# Original test IDs loaded successfully.
# Applying best found multipliers: a=1.98, b=2.18

'submission_final_synthesis.csv' created successfully!


,id,pi_lower,pi_upper
0,200000,822784.689746,1.021477e+06
1,200001,587754.373164,8.132073e+05
2,200002,440741.856504,6.714315e+05
3,200003,279736.628760,4.072457e+05
4,200004,319360.056016,7.650621e+05
